In [ ]:
from typing import Any, Callable, Dict, List, Optional, Union
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from einops import repeat
from diffusers import (
    AudioLDMPipeline,
    AutoencoderKL,
    UNet2DConditionModel,
    DDIMScheduler,
)
from transformers import (
    ClapTextModelWithProjection,
    RobertaTokenizerFast,
    SpeechT5HifiGan,
)

class AudioLDM(nn.Module):
    def __init__(self, device='cuda', repo_id="cvssp/audioldm", config=None):
        super().__init__()
        self.device = torch.device(device)
        pipe = AudioLDMPipeline.from_pretrained(repo_id, use_safetensors=False)

        # Setup components and move to device
        self.pipe = pipe
        self.components = {
            'vae': (pipe.vae, AutoencoderKL),
            'tokenizer': (pipe.tokenizer, RobertaTokenizerFast),
            'text_encoder': (pipe.text_encoder, ClapTextModelWithProjection),
            'unet': (pipe.unet, UNet2DConditionModel),
            'vocoder': (pipe.vocoder, SpeechT5HifiGan),
            'scheduler': (pipe.scheduler, DDIMScheduler)
        }
        
        # Initialize and validate components
        for name, (component, expected_type) in self.components.items():
            if name in ['vae', 'text_encoder', 'unet', 'vocoder']:
                component = component.to(self.device)
            assert isinstance(component, expected_type), f"{name} type mismatch: {type(component)}"
            setattr(self, name, component)

        self.evalmode = True
        self.checkpoint_path = repo_id
        self.audio_duration = 10.24 if not config else config['duration']
        self.original_waveform_length = int(self.audio_duration * self.vocoder.config.sampling_rate)  # 10.24 * 16000 = 163840
        self.vae_scale_factor = 2 ** (len(self.vae.config.block_out_channels) - 1)  # 4
        print(f'[INFO] audioldm.py: loaded AudioLDM!')

In [13]:
from diffusers import AudioLDMPipeline
import torch

repo_id = "cvssp/audioldm"
pipe = AudioLDMPipeline.from_pretrained("cvssp/audioldm", use_safetensors=False, torch_dtype=torch.float32)
pipe = pipe.to("cuda")

prompt = "Foot steps on the wooden floor"
audio = pipe(prompt, num_inference_steps=100, audio_length_in_s=10.24).audios[0]



100%|██████████| 100/100 [00:03<00:00, 33.00it/s]


In [14]:
print(audio.shape)

(163840,)


In [15]:
from IPython.display import Audio

Audio(audio, rate=16000)

In [16]:
import numpy as np

def normalize_wav(data: np.ndarray) -> np.ndarray:
    return data * 0.5

print(audio.min(), audio.max())
# normalized_data = normalize_wav(wav_data)

# print(normalized_data)

-0.45973283 0.4697103


In [ ]:
import scipy

scipy.io.wavfile.write("Foot_steps_on_the_wooden_floor.wav", rate=16000, data=audio)

In [28]:
import torchaudio
wav, ori_sr = torchaudio.load("./Foot_steps_on_the_wooden_floor.wav", normalize=True)  # ts[C,N'±]
wav = wav.numpy()
print(wav.min(), wav.max())

wav2, ori_sr = torchaudio.load("./A_cat_meowing.wav", normalize=True)  # ts[C,N'±]
wav2 = wav2.numpy()
print(wav2.min(), wav2.max())




-0.45973283 0.4697103
-0.46182823 0.39916578


In [30]:
wav_sum = wav + wav2
print(wav_sum.min(), wav_sum.max())
print(wav_sum.dtype)
wav_sum = wav_sum.squeeze()
print(wav_sum.shape)
scipy.io.wavfile.write("Cat_n_Footstep.wav", rate=16000, data=wav_sum)


-0.5448767 0.58386886
float32
(163840,)


In [1]:
from src.data_processing.audio_processing import AudioDataProcessor
import torch

processor = AudioDataProcessor(device='cuda', config_path='configs/audioldm.yaml')
wav = processor.read_wav_file('data/samples/A_cat_meowing.wav')
wav = processor.prepare_wav(wav)
print(wav.shape)
stft, stft_complex = processor.wav_to_stft(wav)
_wav = processor.inverse_stft(stft, stft_complex)

print(_wav.shape)

start assertions
torch.Size([1, 163840])
torch.Size([163840])
(1, 163840)


In [2]:
print(processor.spec_length)

1025


In [3]:
import numpy as np
import torch

wav = wav.cpu().numpy() if isinstance(wav, torch.Tensor) else wav
_wav = _wav.cpu().numpy() if isinstance(_wav, torch.Tensor) else _wav

# 1. Mean Absolute Error (MAE)
mae = np.mean(np.abs(wav - _wav))

# 2. Pearson Correlation Coefficient (PCC)
pcc = np.corrcoef(wav, _wav)[0, 1]

# 3. Normalized Root Mean Squared Error (NRMSE)
rmse = np.sqrt(np.mean((wav - _wav) ** 2))
nrmse = rmse / (np.max(wav) - np.min(_wav) + 1e-8)  # 0 나눔 방지

print(f'MAE: {mae:.4f}, PCC: {pcc:.4f}, NRMSE: {nrmse:.4f}')

MAE: 0.0008, PCC: 1.0000, NRMSE: 0.0014


In [9]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display as ld

def plot_wav_mel(wav_arrays, sr=16000, save_path="./test/result/mel2.png"):
    fig, axes = plt.subplots(2, len(wav_arrays), figsize=(4 * len(wav_arrays), 6))

    clip_duration = 10.24  # 클리핑 길이 (초)
    hop_length = 512       # Hop length 설정

    for i, wav in enumerate(wav_arrays):
        # NumPy 배열 확인 후 1D 변환
        if len(wav.shape) > 1:
            wav = wav.squeeze()

        # 샘플링 레이트 및 오디오 데이터 설정
        duration = len(wav) / sr  # 오디오 길이(초)

        # Clip to first 10.24 seconds if longer
        if duration > clip_duration:
            wav = wav[: int(clip_duration * sr)]  # 앞 10.24초만 유지

        time = np.linspace(0, len(wav) / sr, num=len(wav))

        # **Waveform 플로팅**
        axes[0, i].plot(time, wav, lw=0.5)
        axes[0, i].set_title(f"Waveform {i+1}")
        axes[0, i].set_xlabel("Time (s)")
        axes[0, i].set_ylabel("Amplitude")

        # **Mel Spectrogram 계산 및 플로팅**
        mel_spec = librosa.feature.melspectrogram(y=wav, sr=sr, n_mels=128, hop_length=hop_length)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

        # ✅ specshow()에 전달할 때 (n_mels, frames) 차원 확인
        ld.specshow(mel_spec_db, sr=sr, hop_length=hop_length, x_axis="time", y_axis="mel", ax=axes[1, i])

        axes[1, i].set_title(f"Mel Spectrogram {i+1}")

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()


import torchaudio
wav, ori_sr = torchaudio.load("./data/samples/Cat_n_Footstep.wav", normalize=True)  # ts[C,N'±]
wav = wav.numpy()
print(wav.min(), wav.max())

wav_sep, ori_sr = torchaudio.load("./test/result/cat_separated.wav", normalize=True)  # ts[C,N'±]
wav_sep = wav_sep.numpy()
print(wav_sep.min(), wav_sep.max())

import torchaudio
wav_gt, ori_sr = torchaudio.load("./data/samples/A_cat_meowing.wav", normalize=True)  # ts[C,N'±]
wav_gt = wav_gt.numpy()
print(wav_gt.min(), wav_gt.max())

assert wav.shape == wav_sep.shape, f"wav shape {wav.shape} and _wav shape {wav_sep.shape} must be same"
print(f"wav shape: {wav.shape}")
print(f"wav_sep shape: {wav_sep.shape}")

wav_paths = [
    wav,
    wav_sep,
    wav_gt
]

plot_wav_mel(wav_paths)

-0.5448767 0.58386886
-0.4196167 0.43096924
-0.46182823 0.39916578
wav shape: (1, 163840)
wav_sep shape: (1, 163840)


In [10]:
from src.utils import calculate_sdr, calculate_sisdr

sdr = calculate_sdr(wav_sep, wav_gt)
sisdr = calculate_sisdr(wav_sep, wav_gt)
print(f'SDR: {sdr:.4f}, SI-SDR: {sisdr:.4f}')

SDR: 9.8888, SI-SDR: 10.7649


In [6]:
sdr = calculate_sdr(_wav, _wav)
sisdr = calculate_sisdr(_wav, _wav)
print(f'SDR: {sdr:.4f}, SI-SDR: {sisdr:.4f}')

SDR: 77.1523, SI-SDR: 98.5334


In [7]:
import numpy as np
from scipy.signal import correlate

def align_wav_signals(wav1, wav2):
    assert wav1.shape == wav2.shape, "두 신호의 길이가 같아야 합니다."
    
    # 1D 배열로 변환
    wav1, wav2 = wav1.flatten(), wav2.flatten()
    
    # Cross-Correlation 계산
    correlation = correlate(wav1, wav2, mode="full")
    shift = correlation.argmax() - (len(wav1) - 1)  # 최대 상관점의 오프셋

    # shift 만큼 이동 (앞뒤 zero-padding 적용)
    if shift > 0:
        aligned_wav2 = np.pad(wav2[shift:], (0, shift), mode="constant")
    elif shift < 0:
        aligned_wav2 = np.pad(wav2[:shift], (-shift, 0), mode="constant")
    else:
        aligned_wav2 = wav2  # 이미 정렬됨

    return aligned_wav2, shift

aligned_wav2, applied_shift = align_wav_signals(wav, _wav)
sdr = calculate_sdr(wav, aligned_wav2)
sisdr = calculate_sisdr(wav, aligned_wav2)
print(f"적용된 시간 shift: {applied_shift} samples")
print(f'SDR: {sdr:.4f}, SI-SDR: {sisdr:.4f}')


적용된 시간 shift: 0 samples
SDR: 34.4945, SI-SDR: 49.7118
